# 모델 매개변수 최적화하기
##### 모델과 데이터가 준비되어 있으니, 데이터에 매개변수를 최적화하여 모델을 학습하고, 검증하고, 테스트할 차례이다.
##### 모델을 학습하는 과정은 반복적인 과정이다. 각 반복 단계에서 모델은 출력을 추측하고, 추측과 정답 사이의 오류를 계산하고, 매개변수에 대한 오류의 도함수를 수집한 뒤, 경사하강법을 사용하여 이 파라미터를 최적화한다.

## 기본(Pre-requisite) 코드

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100%|██████████| 26421880/26421880 [00:01<00:00, 15243425.72it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 267286.65it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5006014.76it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 16495245.98it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



## 하이퍼파라미터
##### 하이퍼파라미터는 모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수이다. 서로 다른 하이퍼파라미터 값은 모델 학습과 수렴율에 영향을 미칠 수 있다.
* 에폭 수 - 데이터셋을 반복하는 횟수
* 배치 크기 - 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수
* 학습률 - 각 배치/에폭에서 모델의 매개변수를 조절하는 비율. 값이 작을수록 학습 속도가 느려지고, 값이 크면 학습 중 예측할 수 없는 동작이 발생할 수 있다.

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## 최적화 단계(Optimization Loop)
##### 하이퍼파라미터를 설정한 뒤에는 최적화 단계를 통해 모델을 학습하고 최적화할 수 있다. 최적화 단계의 각 반복을 에폭이라 한다.
##### 하나의 에폭은 아래와 같이 두 부분으로 구성된다.
* 학습 단계 - 학습용 데이터셋을 반복하고 최적의 매개변수로 수렴한다.
* 검증/테스트 단계 - 모델 성능이 개선되고 있는지를 확인하기 위해  테스트 데이터셋을 반복한다.

### 손실 함수(loss function)
##### 학습용 데이터를 제공하면, 학습되지 않은 신경망은 정답을 제공하지 않을 확률이 높다.
##### **손실 함수**는 예측 값과 실제 값 사이의 틀린 정도를 측정하며, 학습 중에 이 값을 최소화할려고 한다. 주어진 데이터 샘플을 입력으로 계산한 예측과 정답을 비교하여 손실(loss)를 계산한다.

##### 일반적인 손실함수에는 회귀 문제에 사용하는 nn.MSELoss(평균 제곱 오차)나 분류에 사용하는 nn.NLLLoss(음의 로그 우도), 그리고 nn.LogSoftmax 와 nn.NLLLoss를 합친 nn.CrossEntropyLoss 등이 있다.
##### 모델의 출력 logits을 nn.CrossEntropyLoss에 전달하여 logits을 정규화하고 예측 오류를 계산한다.

In [3]:
# 손실 함수를 초기화
loss_fn = nn.CrossEntropyLoss()

### 옵티마이저(Optimizer)
##### 최적화는 각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정이다.
##### 최적화 알고리즘은 이 과정(=각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는)이 수행되는 방식을 정의한다.
##### 모든 최적화 로직은 optimizer 객체에 캡슐화된다. 여기서는 SGD 옵티마이저를 사용하고 있으며, PyTorch에는 ADAM, RMSProp과 같은 다른 종류의 모델과 데이터에서 더 잘 동작하는 다양한 옵티마이저가 있다.

In [4]:
# 학습할려는 모델의 매개변수와 학습률 하이퍼파라미터를 등록하여 옵티마이저를 초기화
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

##### 학습 단계에서 최적화는 아래와 같이 3단계로 이뤄진다.
* optimizer.zero_grad()를 호출하여 모델 매개변수의 변화도를 재설정한다. 기본적으로 변화도는 더해지기 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정한다.
* loss.backward()를 호출하여 예측 손실을 역전파한다. PyTorch는 각 매개변수에 대한 손실의 변화도를 저장한다.
* 변화도를 계산한 뒤에는 optimizer.step()을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정한다.

## 전체 구현
##### 최적화 코드를 반복하여 수행하는 train_loop와 테스트 데이터로 모델의 성능을 측정하는 test_loop로 정의

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X,y) in enumerate(dataloader):
    # 예측과 loss 계산
    pred = model(X)
    loss = loss_fn(pred, y)

    # 역전파
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1) * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  # 단순 순전파 계산을 위해 torch.no_grad()
  with torch.no_grad():
    for X, y in dataloader:
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      # 최대값을 가지는 인덱스 = y 요소의 합
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

##### 손실 함수와 옵티마이저를 초기화하고 train_loss 와 test_loss에 전달한다.

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.291444  [   64/60000]
loss: 2.280236  [ 6464/60000]
loss: 2.260664  [12864/60000]
loss: 2.265734  [19264/60000]
loss: 2.242474  [25664/60000]
loss: 2.217055  [32064/60000]
loss: 2.223252  [38464/60000]
loss: 2.184766  [44864/60000]
loss: 2.188356  [51264/60000]
loss: 2.158250  [57664/60000]
Test Error: 
 Accuracy: 43.7%, Avg loss: 2.148204 

Epoch 2
-------------------------------
loss: 2.163132  [   64/60000]
loss: 2.144088  [ 6464/60000]
loss: 2.090827  [12864/60000]
loss: 2.107517  [19264/60000]
loss: 2.051038  [25664/60000]
loss: 2.002467  [32064/60000]
loss: 2.020483  [38464/60000]
loss: 1.944626  [44864/60000]
loss: 1.957958  [51264/60000]
loss: 1.879049  [57664/60000]
Test Error: 
 Accuracy: 50.0%, Avg loss: 1.873589 

Epoch 3
-------------------------------
loss: 1.918590  [   64/60000]
loss: 1.873711  [ 6464/60000]
loss: 1.763783  [12864/60000]
loss: 1.800990  [19264/60000]
loss: 1.692506  [25664/60000]
loss: 1.654520  [32064/600

TEST

In [7]:
test_loss, correct = 0, 0

for X, y in train_dataloader:
  pred = model(X)
  test_loss += loss_fn(pred, y).item()
  correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  print(pred)
  # 최대값을 가지는 인덱스 반환
  print(pred.argmax(1))
  break

tensor([[-5.2892e+00, -8.5947e+00, -3.1694e+00, -3.0237e+00, -1.4874e+00,
          6.5159e+00, -2.6852e+00,  4.5896e+00,  4.6271e+00,  8.5195e+00],
        [ 7.6042e+00, -9.5918e-01,  2.6316e+00,  3.5016e+00,  1.0127e+00,
         -5.2327e+00,  4.6072e+00, -9.1702e+00, -4.4192e-01, -4.6143e+00],
        [ 1.4757e+00,  2.5883e+00,  2.5685e-01,  2.5484e+00,  8.7223e-01,
         -2.0115e+00,  9.1538e-01, -3.3640e+00, -1.6134e+00, -2.3397e+00],
        [ 3.5244e+00,  2.2243e+00,  1.6739e+00,  2.8681e+00,  1.4790e+00,
         -3.6319e+00,  2.4234e+00, -5.9630e+00, -1.6346e+00, -4.0305e+00],
        [ 3.0613e+00,  3.8625e+00, -1.8111e-01,  5.4474e+00,  1.3131e+00,
         -4.2237e+00,  1.5888e+00, -6.4711e+00, -1.8009e+00, -4.1080e+00],
        [ 1.9154e+00, -2.2373e+00,  5.3237e+00,  5.8161e-02,  4.1175e+00,
         -3.2682e+00,  4.2346e+00, -7.3644e+00,  8.4527e-01, -4.8341e+00],
        [-4.0816e+00, -4.4500e+00, -2.4722e+00, -1.7373e+00, -1.6095e+00,
          4.7541e+00, -2.1929e+0